In [1]:
from opencage.geocoder import OpenCageGeocode
from opencage.geocoder import InvalidInputError, RateLimitExceededError, UnknownError
import pandas as pd
from pprint import pprint

In [6]:
# key = '44064484207b4b37b2427a3435b40b07'
# key = '26663d3211884ff0ac57447c975865f0'
# key = '497ad12427ef409ab854af53608282b4'
key = '2c67f66fd0cd44cdacf36b54154b9cdf'
geocoder = OpenCageGeocode(key)

In [3]:
df = pd.read_csv("data/eclipse_data_enriched_5000_years.csv")

In [43]:
def reverse_geocode(row):  
    latitude = row['Eclipse Latitude']
    longitude = row['Eclipse Longitude'] 
    try:
        results = geocoder.reverse_geocode(latitude, longitude)
        if results and len(results):
            return get_location(results)
    except RateLimitExceededError as ex:
        return str(ex)
    except InvalidInputError as ex:
        return str(ex)
    
    return None, None, None

def get_location(results):
    location = results[0]['components']

    country_name = location.get('country', None)
    alpha_2_code = location.get('ISO_3166-1_alpha-2', None)
    alpha_3_code = location.get('ISO_3166-1_alpha-3', None)

    return country_name, alpha_2_code, alpha_3_code 

def update_csv(df, filename, batch_size=100):
    for i in range(7252, 7252+2500, batch_size):
        if(i % 50) == 0:
            print(i)
        batch = df[i:i+batch_size]
        # print(zip(*batch.apply(reverse_geocode, axis=1)))
        batch['Country'], batch['Alpha2'], batch['Alpha3'] = zip(*batch.apply(reverse_geocode, axis=1))
        # if 'Country' in df.columns:
        #     df.drop(columns=['Country'], inplace=True)
        batch.to_csv(filename, mode='a', header=not i, index=False)

In [8]:
# lat = 44.8303087
# long = -0.5761911

# c, a2, a3 = reverse_geocode(lat, long)

In [11]:
# c, a2, a3

('France', 'FR', 'FRA')

In [ ]:
update_csv(df, 'data/geo_coded_data.csv', batch_size=100)

In [2]:
df_with_country = pd.read_csv("data/geo_coded_data.csv")

In [15]:
df_with_country.to_csv("data/geo_coded_data.csv", index = False)

CREATE JSON WITH COUNTRY NAME TO ALPHA3 CODE MAPPING

In [49]:
import json

# Load the JSON data
with open('data/world.geojson', 'r') as f:
    data = json.load(f)

# Create a dictionary to store the mapping
country_mapping = {}

# Iterate through the features
for feature in data['features']:
    country_name = feature['properties']['name']
    country_id = feature['id']
    country_mapping[country_name] = country_id

# Write the mapping to a new JSON file
with open('data/alpha3Codes.json', 'w') as f:
    json.dump(country_mapping, f, indent=4)


ADDING CODES TO DATA

In [50]:
data = pd.read_csv("data/geo_coded_data.csv")

In [54]:
import json

# Read the JSON file
with open("data/alpha3Codes.json", "r") as f:
    country_mapping = json.load(f)
    

# Function to get Alpha3Code
def get_alpha3code(country_name):
    alpha3code = country_mapping.get(country_name, "NOT FOUND")
    if alpha3code == "NOT FOUND":
        print(f"Country not found: {country_name}")
    return alpha3code

In [ ]:
# Add Alpha3Code column
data["Alpha3Code"] = data["Country"].apply(lambda x: get_alpha3code(x) if pd.notnull(x) else None)

CLEAN GEOCODED DATA

In [55]:
data = pd.read_csv("data/geo_coded_data.csv")

In [48]:
data = data[:8961]

In [56]:
data = data[data['Country'].notna() & data['Alpha3'].notna()]

In [57]:
len(data)

575

In [58]:
data.to_csv("data/geo_coded_data_clean.csv", index = False)